1. Get the Data(pre-process it)
2. Pick an algorithm
3. Train the Algorithm,Verify accuracy.Optimize<br>
4. Predict

# Gunbir Singh

In [1]:
import pandas as pd
import numpy as np
import math
import random
import warnings
from collections import Counter

In [2]:
#Read breast-cancer-wisconsin.data Data set

In [3]:
df=pd.read_csv('breast-cancer-wisconsin.csv',delimiter=',')
df.replace('?',-99999,inplace=True)


-99999 is a huge outlier so we replace ? with it

In [4]:
df.head()

,id,sample_code_number,cell_size,cell_shape,marginal_adhesion,epithelial_cell_size,bare_nuclei,bland_chromation,normal_nucleoli,mitoses,class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2


In [5]:
df.columns = ['id','sample_code_number', 'cell_size','cell_shape','marginal_adhesion','epithelial_cell_size','bare_nuclei','bland_chromation','normal_nucleoli','mitoses','class']

In [6]:
df.head()

,id,sample_code_number,cell_size,cell_shape,marginal_adhesion,epithelial_cell_size,bare_nuclei,bland_chromation,normal_nucleoli,mitoses,class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2


In [7]:
df = df.drop(['id'],axis=1)

In [8]:
df.head()

,sample_code_number,cell_size,cell_shape,marginal_adhesion,epithelial_cell_size,bare_nuclei,bland_chromation,normal_nucleoli,mitoses,class
0,5,1,1,1,2,1,3,1,1,2
1,5,4,4,5,7,10,3,2,1,2
2,3,1,1,1,2,2,3,1,1,2
3,6,8,8,1,3,4,3,7,1,2
4,4,1,1,3,2,1,3,1,1,2


In [9]:
df.describe()

,sample_code_number,cell_size,cell_shape,marginal_adhesion,epithelial_cell_size,bland_chromation,normal_nucleoli,mitoses,class
count,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000
mean,4.417740,3.134478,3.207439,2.806867,3.216023,3.437768,2.866953,1.589413,2.689557
std,2.815741,3.051459,2.971913,2.855379,2.214300,2.438364,3.053634,1.715078,0.951273
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000
25%,2.000000,1.000000,1.000000,1.000000,2.000000,2.000000,1.000000,1.000000,2.000000
50%,4.000000,1.000000,1.000000,1.000000,2.000000,3.000000,1.000000,1.000000,2.000000
75%,6.000000,5.000000,5.000000,4.000000,4.000000,5.000000,4.000000,1.000000,4.000000
max,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,4.000000


In [10]:
df.shape

(699, 10)

In [11]:
#just to make sure evrything in our data set is in int or float
full_data = df.astype(float).values.tolist()
#print(full_data[:5])
#Shuffle the data 
random.shuffle(full_data)


In [12]:
def k_nearest_neighbors(train_set_data, predict, k):
    if len(train_set_data) >= k:
        warnings.warn('K is less than total number of set_datas')
    distances = []
    for set_data in train_set_data:
        for features in train_set_data[set_data]:
            euclidean_distance = np.sqrt(np.sum((np.array(features)-np.array(predict))**2))
            distances.append([euclidean_distance, set_data])
    #print(distances)
    votes = [i[1] for i in sorted(distances)[:k]]
    vote_result = Counter(votes).most_common(1)[0][0]
    
    return vote_result

In [13]:
split_size = 0.3
#Dictionaries
train_set = {2:[], 4:[]}
test_set = {2:[], 4:[]}
#print(full_data)
split=int(split_size*len(full_data))
#First 60% data
train_data = full_data[:-split]
#print(train_data)
#The last 40% of the data
test_data = full_data[-split:]

In [14]:
for i in train_data: 
    #i[-1] is the class which is either 2 or 4 and we wouldn't want that in our training set
    train_set[i[-1]].append(i[:-1])

In [15]:
for i in test_data:
    test_set[i[-1]].append(i[:-1])


In [16]:
match = 0
total = 0
#i is either 2 or 4
for i in test_set:
    for appended_data in test_set[i]:
        vote = k_nearest_neighbors(train_set, appended_data, k=3)
        if(i == vote):
            match = match+1
        total = total+1
        accuracy = match/total
        
print('Accuracy for this algorithm:',accuracy)

Accuracy for this algorithm: 0.9665071770334929
